In [1]:
file = open("fileToCorrupt.txt", "wb")
string = b"This is some cool text that is unfortunately going to be corrupted. :("
file.write(string)

len(string)

file.close()

In [2]:
import os
from Crypto.Cipher import AES
import binascii

def pad(inString, blockSize):
    remainder = blockSize - (len(inString) % blockSize)
    print("Remainder is: " + str(remainder))
    paddedString = inString +  bytes([remainder] * remainder)
    return paddedString



key = b"secretkeeeeeeeey"
print("Key is: " + str(key))
print("Len of key is: " + str(len(key)))

ecbEncryptor = AES.new(key, AES.MODE_ECB)
cbcEncryptor = AES.new(key, AES.MODE_CBC, b"secretvectooooor")
    

file = open("fileToCorrupt.txt", "rb")
contents = file.read()




Key is: b'secretkeeeeeeeey'
Len of key is: 16


In [3]:
print("Read in: " + str(binascii.hexlify(contents)))
print("Which is: " + str(contents))
print("Len of raw: " + str(len(contents)))
paddedString = pad(contents, 16)
print("Padded to 16: " + str(binascii.hexlify(paddedString)))
print("Which is: " + str(paddedString))
print("Len of padded contents: " +str(len(paddedString)))

ecb_encryptedContents = ecbEncryptor.encrypt(paddedString)
cbc_encryptedContents = cbcEncryptor.encrypt(paddedString)

print("Encrypted: " + str(binascii.hexlify(ecb_encryptedContents)))
print("Encrypted: " + str(binascii.hexlify(cbc_encryptedContents)))

Read in: b'5468697320697320736f6d6520636f6f6c2074657874207468617420697320756e666f7274756e6174656c7920676f696e6720746f20626520636f727275707465642e203a28'
Which is: b'This is some cool text that is unfortunately going to be corrupted. :('
Len of raw: 70
Remainder is: 10
Padded to 16: b'5468697320697320736f6d6520636f6f6c2074657874207468617420697320756e666f7274756e6174656c7920676f696e6720746f20626520636f727275707465642e203a280a0a0a0a0a0a0a0a0a0a'
Which is: b'This is some cool text that is unfortunately going to be corrupted. :(\n\n\n\n\n\n\n\n\n\n'
Len of padded contents: 80
Encrypted: b'4ade7c5b2116495335be1173ba93f0ae81ae4a1031ab16e1fa312dd8f7949b4e6714cf13889018ae8ad78262a32b19343f91436868faf85d9eb0c1181c1bc210efff1a811380424cfe4c15f7af86d821'
Encrypted: b'1de7d50e676c02880a3b93c8db31ab00aa888cb778d6d293d4c0a9747e19a9562da1a4228652d9b083f907cbb51c96ce30dc59d35fd82599b7b8f895b7dcda5106d2af6bb034e95eb74d31b24ad41636'


In [4]:
# Now to corrupt a byte

byteToCorrupt = 30
ecb_corruptedContents = ecb_encryptedContents[0:byteToCorrupt - 1] + bytes([int(ecb_encryptedContents[byteToCorrupt]) + 1]) + ecb_encryptedContents[byteToCorrupt:]
cbc_corruptedContents = cbc_encryptedContents[0:byteToCorrupt - 1] + bytes([int(cbc_encryptedContents[byteToCorrupt]) + 1]) + cbc_encryptedContents[byteToCorrupt:]
print("ECB Corrupted: " + str(binascii.hexlify(ecb_encryptedContents)))
print("CBC Corrupted: " + str(binascii.hexlify(cbc_encryptedContents)))

ECB Corrupted: b'4ade7c5b2116495335be1173ba93f0ae81ae4a1031ab16e1fa312dd8f7949b4e6714cf13889018ae8ad78262a32b19343f91436868faf85d9eb0c1181c1bc210efff1a811380424cfe4c15f7af86d821'
CBC Corrupted: b'1de7d50e676c02880a3b93c8db31ab00aa888cb778d6d293d4c0a9747e19a9562da1a4228652d9b083f907cbb51c96ce30dc59d35fd82599b7b8f895b7dcda5106d2af6bb034e95eb74d31b24ad41636'


In [5]:
def depad(inString):
    length = inString[-1]
    for i in range(-1, -length, -1):
        if inString[i] != length:
            return inString
    return inString[:-length]

cbcDecryptor = AES.new(key, AES.MODE_CBC, b"secretvectooooor")

ecb_decryptedContents = ecbEncryptor.decrypt(ecb_corruptedContents)
cbc_decryptedContents = cbcDecryptor.decrypt(cbc_corruptedContents)
print("ECB Decrypted: " + str(depad(ecb_decryptedContents)))
print("CBC Decrypted: " + str(depad(cbc_decryptedContents)))

ECB Decrypted: b"This is some cool'\x9cx\xb5|\x12\x1f\xb8IN\x07\x9b\xe1\x1a\xcfnfortunately going to be corrupted. :("
CBC Decrypted: b'This is some coo\xba\xd2\xae\xf9\xf0\xd4\xaa\xb3@\xe1\xe9\x18\xe6\x8f\xce\x0enfortunately \xd4oing to be corrupted. :('
